In [ ]:
!pip install qiskit
!pip install qiskit_aer
!pip install pylatexenc
!pip install qutip
!pip install pennylane


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=caa622ab44efe74f8406312a98ce825406fad87760f7f0ef0c0f13a098146508
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pennylane as qml
from pennylane import numpy as np

/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


## Toy Example: The Repetition Code

To start with, let us explore the general structure of error-correction codes using a simple example: the **three-qubit repetition code**. We introduce this code as a quantum circuit with definite steps to build intuition about how it corrects qubit errors.  In this formalism, error-correction codes follow a simple structure:

1. **Qubit encoding**  
2. **Error detection**  
3. **Error correction**

### Qubit Encoding

The first step in any error-correction code is to encode one abstract (logical) qubit into multiple physical qubits. The idea is that if some external disturbance affects one of the qubits, the remaining qubits still contain enough information to recover the original logical qubit.

In the three-qubit repetition code, the logical basis states (logical codewords)  
$|0_L\rangle$ (“logical 0”) and $|1_L\rangle$ (“logical 1”)  
are encoded into three physical qubits as

\begin{align}
|0_L\rangle = |000\rangle,\qquad
|1_L\rangle = |111\rangle.\nonumber
\end{align}

A general qubit  
\begin{align}
|\psi\rangle = \alpha|0\rangle + \beta|1\rangle\nonumber
\end{align}
is then encoded as

\begin{align}
|\psi_L\rangle = \alpha|000\rangle + \beta|111\rangle.\nonumber
\end{align}

This encoding can be implemented using the following quantum circuit.

<img src='http://drive.google.com/uc?export=download&id=15AwWdCx1qiFxa57UiTlyxCS7nIfEM02Y' width="800" height="320"/>

In [ ]:
def encode(alpha, beta):
    qml.StatePrep([alpha, beta], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])


def encoded_state(alpha, beta):
    encode(alpha, beta)
    return qml.state()


encode_qnode = qml.QNode(encoded_state, qml.device("default.qubit"))

alpha = 1 / np.sqrt(2)
beta = 1 / np.sqrt(2)

encode_qnode = qml.QNode(encoded_state, qml.device("default.qubit"))

print("|000> component: ", encode_qnode(alpha, beta)[0])
print("|111> component: ", encode_qnode(alpha, beta)[7])

|000> component:  (0.7071067811865475+0j)
|111> component:  (0.7071067811865475+0j)


### Note
<span style="color:red;">*Why do we encode qubits in this way instead of preparing many copies of the state?*</span>

If the quantum state is **known**, we *could* prepare multiple copies — but this only increases the required quantum resources and offers no real benefit.  
If the quantum state is **unknown**, the <span style="color:red;">*no-cloning theorem*</span>
 states that it is fundamentally impossible to create a copy of an arbitrary unknown quantum state. Hence, encoding must be done through entanglement, not copying.

---

## Error Detection

Now suppose a **bit-flip error** occurs on the second qubit, meaning that the qubit is randomly flipped. This can be modeled as an unwanted Pauli-$X$ operator acting on the second qubit.
\begin{align}
X_2 \left( \alpha|000\rangle + \beta|111\rangle \right)
= \alpha|010\rangle + \beta|101\rangle.\nonumber
\end{align}

How do we detect this error? As we know, directly measuring the qubit collapses the quantum state, so we cannot simply measure the data qubits to detect the error.

To detect a bit-flip error on one of the physical qubits **without disturbing the encoded logical state**, we use a **parity measurement**. This checks whether all physical qubits are in the same state by comparing them two at a time, without directly measuring them. Instead, *auxiliary qubits* (ancillas) are used and measured.

For the three-qubit repetition code, this involves measuring two ancilla qubits in the computational basis after applying a sequence of CNOT gates, as shown in the circuit below. The ancilla qubits record whether each pair of data qubits are the same or different:

- If the pair is the **same**, the ancilla remains in state \(0\).  
- If the pair is **different**, the ancilla flips to state \(1\).

Thus, if all physical qubits are identical, both ancillas remain at \(0\).  
This procedure constitutes the **parity measurement**.
<img src="http://drive.google.com/uc?export=download&id=1afk4vpTSIM4SGLdw5cFyVHZxuxc49ZTY" width="600" height="320"/>

The result of these parity measurements is known as the **syndrome**.  
Since we use two auxiliary qubits, there are four possible measurement outcomes:  
**00, 01, 10, 11**.

Each outcome indicates whether a bit-flip error occurred in the encoded state, and if so, **which physical qubit** was flipped.  
The table below summarizes how to interpret the error from the syndrome.
<img src="http://drive.google.com/uc?export=download&id=13kfFa1aAhoJ386a2AL1sFj4iMUrtul_O" width="720" height="320"/>

When there is no error, the syndrome measurement will yield $0$ on both auxiliary qubits. Let us verify the full table by implementing the syndrome measurement in PennyLane.

In [ ]:
def error_detection():
    qml.CNOT(wires=[0, 3])
    qml.CNOT(wires=[1, 3])
    qml.CNOT(wires=[1, 4])
    qml.CNOT(wires=[2, 4])


dev = qml.device("default.qubit", wires=5, shots=1)# A single sample flags error
@qml.qnode(dev)
# @qml.qnode(qml.device("default.qubit", wires=5))
def syndrome_measurement(error_wire):
    encode(alpha, beta)

    qml.PauliX(wires=error_wire)  # Unwanted Pauli Operator

    error_detection()

    return qml.sample(wires=[3, 4])


print("Syndrome if error on wire 0: ", syndrome_measurement(0))
print("Syndrome if error on wire 1: ", syndrome_measurement(1))
print("Syndrome if error on wire 2: ", syndrome_measurement(2))

Syndrome if error on wire 0:  [[1 0]]
Syndrome if error on wire 1:  [[1 1]]
Syndrome if error on wire 2:  [[0 1]]


/usr/local/lib/python3.12/dist-packages/pennylane/devices/device_api.py:193: PennyLaneDeprecationWarning: Setting shots on device is deprecated. Please use the `set_shots` transform on the respective QNode instead.
  warnings.warn(


## Error Correction

Once a single bit-flip error is detected, correction is straightforward.  
Since the Pauli-$X$ operator is its own inverse (i.e., $X^2 = I$),  
reapplying the $X$ operator to the erroneous qubit restores the original encoded state.

For example, if the syndrome indicates that the error occurred on the second qubit  
(qubits are labeled 0, 1, and 2), we apply

\begin{align}
X_1(X_1(|\psi_L\rangle))=|\psi_L\rangle\nonumber
\end{align}

to correct it.

By applying the appropriate corrective operation, the repetition code protects and repairs the encoded quantum information.  
The full workflow is illustrated in the circuit below.


<img src="http://drive.google.com/uc?export=download&id=12LfD24w_z0nFve8TG-QUDootjdRLugoM" width="820" height="320"/>


In [ ]:
@qml.qnode(qml.device("default.qubit", wires=5))
def error_correction(error_wire):
    encode(alpha, beta)

    qml.PauliX(wires=error_wire)

    error_detection()

    # Mid circuit measurements

    m3 = qml.measure(3)
    m4 = qml.measure(4)

    # Operations conditional on measurements

    qml.cond(m3 & ~m4, qml.PauliX)(wires=0)
    qml.cond(m3 & m4, qml.PauliX)(wires=1)
    qml.cond(~m3 & m4, qml.PauliX)(wires=2)

    return qml.density_matrix(
        wires=[0, 1, 2]
    )  # qml.state not supported, but density matrices are.

In [ ]:
dev = qml.device("default.qubit", wires=5)
error_correction_qnode = qml.QNode(error_correction, dev)
encoded_state = qml.math.dm_from_state_vector(encode_qnode(alpha, beta))

# Compute fidelity of final corrected state with initial encoded state

print(
    "Fidelity when error on wire 0: ",
    qml.math.fidelity(encoded_state, error_correction_qnode(0)).round(2),
)
print(
    "Fidelity when error on wire 1: ",
    qml.math.fidelity(encoded_state, error_correction_qnode(1)).round(2),
)
print(
    "Fidelity when error on wire 2: ",
    qml.math.fidelity(encoded_state, error_correction_qnode(2)).round(2),
)

Fidelity when error on wire 0:  1.0
Fidelity when error on wire 1:  1.0
Fidelity when error on wire 2:  1.0


## Revisiting the Three-Qubit Repetition Code with Pauli Operators

So far, we have worked with a simple example — the three-qubit repetition code — but it is quite limited.  
This code can correct only a **single bit-flip error**. More powerful codes exist, but they also require more resources.  
For instance, **Shor’s code** can correct *any* single-qubit error, but it needs **9 physical qubits**.  
In realistic fault-tolerant architectures, one may even need **around 1000 physical qubits per logical qubit** to suppress errors to acceptable levels.

As the number of qubits grows, the encoded states and protocols become increasingly complex.  
Representing a state vector of 1000 qubits is clearly impractical. To manage such situations, we move to a different representation of quantum error-correction codes: **Pauli-operator descriptions** rather than state-vector descriptions.

### Note
If you are not yet familiar with Pauli operators and their key properties, this is an excellent time to review them.

To build some intuition for the operator picture, let us rewrite the three-qubit repetition code using Pauli operators.  
Using the identity below,


<img src="http://drive.google.com/uc?export=download&id=1WK6JafejJT_Zo1hPp6A-7wdRiOf2qCGr" width="520" height="320"/>

the three-qubit repetition code can be expressed in the following way.

<img src="http://drive.google.com/uc?export=download&id=17BUjMerAKT_itS7qetax0qlmzosLlkxH" width="820" height="320"/>

This is the same circuit, but now the controls are all on the auxiliary qubits, while the logical (data) qubits act as the targets. At first glance, this seems undesirable — we do **not** want to change the state of the logical qubits!  

However, notice that the operators acting on the logical qubits are $Z_1 Z_2$ and $Z_2 Z_3$, both of which leave the logical codewords invariant.

This is good news: the operators $Z_1 Z_2$ and $Z_2 Z_3$ leave the logical states unchanged.  
Any state of the form  
\begin{align}
|\psi_L\rangle = \alpha |000\rangle + \beta |111\rangle\nonumber
\end{align}
is a $+1$ eigenstate of both operators, meaning the entire logical code space is invariant under their action.

A single bit-flip error breaks this invariance: at least one of the operators will return an eigenvalue of $-1$.  
Thus, measuring these two operators reveals whether an error occurred and, as shown in the syndrome table below, identifies which qubit experienced the error.

In this sense, detecting and correcting errors requires only two Pauli operators and the outcomes of their eigenvalue measurements.


<img src="http://drive.google.com/uc?export=download&id=1cWK6qvWf5AQFQdPPPmhrmnqFyPlTEnCa" width="520" height="320"/>

This table is closely related to the previous syndrome table. If we know that the initial state was $|\psi_L\rangle$, and we assume that only a single bit-flip occurred (i.e., the state becomes one of the states with two zeros and one one, or vice versa), then the pair of measured eigenvalues uniquely identifies the erroneous state. Consequently, we can determine **which qubit** was flipped.

This perspective gives us a new way to characterize error-correction codes.  
Instead of starting with codewords and then figuring out the corresponding syndrome-measurement operators, we can reverse the process. That is, we may **begin by specifying a set of operators**, find the states that remain invariant under their action, and declare those states to be our **codewords**. The operators in this initial set are called **stabilizer generators**.

---

## The Stabilizer Formalism

### Stabilizer Generators

---

### Groups

To understand the stabilizer formalism, it is helpful to recall some basic group theory.  
A **group** is a set equipped with:

1. **A binary operation:** combining any two elements $a$ and $b$ yields another element in the set (e.g., $c = a + b$).  
2. **An identity element** $e$ such that $e + a = a$ for any element $a$.  
3. **An inverse** $-a$ for each element $a$, such that $a + (-a) = e$.

---

### Notation

In the stabilizer formalism, tensor-product symbols ($\otimes$) are often omitted.  
For example, $XZ$ denotes $X \otimes Z$ acting on qubits 0 and 1, respectively.  
When identity operators are omitted, subscripts are used to show which qubits the non-identity Pauli operators act on.  

If all positions are filled, such as $XZI$, then the operator acts on qubits 0, 1, and 2 in that order.

---

## Stabilizer Groups

The stabilizer formalism provides a powerful framework for constructing quantum error-correcting codes using the algebraic structure of Pauli operators.

It works with subgroups of the **Pauli group on $n$ qubits**, denoted $\mathcal{P}_n$, which consists of all tensor products of single-qubit Pauli operators $\{I, X, Y, Z\}$, up to overall phases $\{\pm 1, \pm i\}$.

A **stabilizer group** $\mathcal{S}$ is a subgroup of $\mathcal{P}_n$ that satisfies:

1. It contains the identity operator $I$.  
2. All elements of $\mathcal{S}$ **commute**.  
3. The product of any two elements is also in $\mathcal{S}$.  
4. It does **not** contain the negative identity operator $-I$.

Instead of listing every element of $\mathcal{S}$, we describe it through a set of **generators**: a minimal set of operators whose finite products generate the entire stabilizer group.

For example, consider the stabilizer set:

\begin{align}
\mathcal{S} = \{I, Z_0 Z_1, Z_1 Z_2, Z_0 Z_2\}.\nonumber
\end{align}

You can check that it satisfies properties 1–4 above.  
The most tedious property to verify is closure (property 3), where we must check that all possible products of elements remain in $\mathcal{S}$.  
For example,

\begin{align}
(Z_0 Z_1)(Z_1 Z_2) = Z_0 Z_2,\nonumber
\end{align}

and so on.  
We see that all elements of $\mathcal{S}$ can be produced from the generators $Z_0 Z_1$ and $Z_1 Z_2$. Thus, these are **stabilizer generators** for $\mathcal{S}$, which we write as

\begin{align}
\mathcal{S} = \langle Z_0 Z_1,\; Z_1 Z_2 \rangle,\nonumber
\end{align}

read as: “$\mathcal{S}$ is the stabilizer group generated by $Z_0 Z_1$ and $Z_1 Z_2$.”

Specifying the generators is therefore sufficient to fully define both the stabilizer group and the corresponding quantum error-correcting code.

---

Now that we understand stabilizer generators, let us build a tool (to be used later) that constructs the full stabilizer group from its generators.


In [ ]:
import itertools
from pennylane import X, Y, Z
from pennylane import Identity as I


def generate_stabilizer_group(gens, num_wires):
    group = []
    init_op = I(0)
    for i in range(1, num_wires):
        init_op = init_op @ I(i)
    for bits in itertools.product([0, 1], repeat=len(gens)):
        op = init_op
        for i, bit in enumerate(bits):
            if bit:
                op = qml.prod(op, gens[i]).simplify()
        group.append(op)
    return set(group)


generators = [Z(0) @ Z(1) @ I(2), I(0) @ Z(1) @ Z(2)]
generate_stabilizer_group(generators, 3)

{I(0) @ I(1) @ I(2), Z(0) @ Z(1), Z(0) @ Z(2), Z(1) @ Z(2)}

## 🌟 Defining the Codespace

Indeed, by inputting only the **generators**, we obtain *all* the elements of the stabilizer group.  
Feel free to experiment with the code using different generator sets!

---

### 🔹 What Does It Mean to Define a Code?

Given a collection of stabilizer generators—just Pauli strings obeying certain algebraic properties—we recall the central requirement:

> **Every stabilizer must leave every codeword invariant.**

For any stabilizer element \( S \) and codeword \( |\psi\rangle \), we must have  
\begin{align}
S|\psi\rangle = |\psi\rangle.\nonumber
\end{align}

---

### 🔹 The Codespace

The **codespace** is the set of all quantum states that satisfy  
\begin{align}
S|\psi\rangle = |\psi\rangle \quad \text{for all } S \in \mathcal{S},\nonumber
\end{align}
where \( \mathcal{S} \) is the stabilizer group.

Once the codespace is defined, the **codewords** are simply any orthogonal basis spanning this space.  
For instance, for the **three-qubit repetition code**, the codewords \( |0_L\rangle \) and \( |1_L\rangle \) arise from stabilizers  
\begin{align}
Z_1 Z_2,\qquad Z_2 Z_3.\nonumber
\end{align}

A remarkable fact:

> **There is a one-to-one correspondence between stabilizer groups and the quantum codes they define.**

Thus, a quantum error-correcting code can be fully described *either* by listing its codewords *or* by specifying its stabilizer generators.

---

## 🌟 Logical Operators

We now turn to **logical operators**—operations that act on the encoded qubits.

Stabilizer generators define *syndrome measurements*, and codewords lie in the space fixed by every stabilizer. Logical operators must:

1. Act **non-trivially** on the codewords (so they are *not* stabilizers),  
2. But still **preserve the codespace**, meaning they must commute with every stabilizer generator.

If a logical operator failed to commute with a stabilizer, it would map valid states out of the codespace or alter syndromes, corrupting the encoded information.

We are particularly interested in the **logical Pauli operators**, \( X_L \) and \( Z_L \), defined by
\begin{align}
X_L |0_L\rangle = |1_L\rangle,\qquad\nonumber\\
Z_L |0_L\rangle = |0_L\rangle,\qquad\nonumber\\
Z_L |1_L\rangle = -|1_L\rangle.\nonumber
\end{align}

As an example, in the **three-qubit bit-flip code**, the logical operators are:
\begin{align}
X_L = X_1 X_2 X_3, \qquad Z_L = Z_1.\nonumber
\end{align}

In general, for a stabilizer set \( \mathcal{S} \), the logical operators must satisfy:

- **Commutation:** they commute with all elements of \( \mathcal{S} \),
- **Nontriviality:** they are *not* themselves in \( \mathcal{S} \),
- **Anticommutation:** logical \( X_L \) and \( Z_L \) anticommute.

---

## 🌟 The LSD Theorem

The stabilizer group is a subgroup of the Pauli group, and every Pauli operator interacts with it in a structured way.  
The **LSD theorem** states that Pauli operators can be classified into three distinct types:

- **L — Logical operators:**  
  commute with all stabilizers but act non-trivially on the codewords.

- **S — Stabilizers:**  
  leave all codewords unchanged.

- **D — Destabilizers (errors):**  
  anticommute with at least one stabilizer and push the state out of the codespace.

This classification is crucial for distinguishing:

- which operators are harmless (S),  
- which implement encoded computation (L), and  
- which represent errors to be corrected (D).

---

## 🌟 Next Step: Coding the LSD Classifier

We will now write code that, given a set of stabilizer generators, **classifies any Pauli operator** according to the LSD theorem.

Let’s proceed to the implementation!


In [ ]:
def classify_pauli(operator, logical_ops, generators, n_wires):
    allowed_wires = set(range(n_wires))
    operator_wires = set(operator.wires)

    assert operator_wires.issubset(allowed_wires), (
        "Operator has wires not allowed by the code"
    )

    operator_names = set([op.name for op in operator.decomposition()])
    allowed_operators = set(["Identity", "PauliX", "PauliY", "PauliZ", "SProd"])

    assert operator_names.issubset(allowed_operators), (
        "Operator contains an illegal operation"
    )

    stabilizer_group = generate_stabilizer_group(generators, n_wires)

    if operator.simplify() in stabilizer_group:
        return f"{operator} is a Stabilizer."

    if all(qml.is_commuting(operator, g) for g in generators):
        if operator in logical_ops:
            return f"{operator} is a Logical Operator."
        else:
            return f"{operator} commutes with all stabilizers — it's a Logical Operator (or a multiple of one)."

    return f"{operator} is an Error Operator (Destabilizer)."


generators = [Z(0) @ Z(1) @ I(2), I(0) @ Z(1) @ Z(2)]
logical_ops = [X(0) @ X(1) @ X(2), Z(0) @ Z(1) @ Z(2)]
print(classify_pauli(Z(0) @ I(1) @ Z(2), logical_ops, generators, 3))
print(classify_pauli(Y(0) @ Y(1) @ Y(2), logical_ops, generators, 3))
print(classify_pauli(X(0) @ Y(1) @ Z(2), logical_ops, generators, 3))

Z(0) @ I(1) @ Z(2) is a Stabilizer.
Y(0) @ Y(1) @ Y(2) commutes with all stabilizers — it's a Logical Operator (or a multiple of one).
X(0) @ Y(1) @ Z(2) is an Error Operator (Destabilizer).


## Example: Five-qubit stabilizer code

The 5-qubit code, also called as Laflamme’s code, holds a special place as the smallest error correcting code capable of correcting arbitrary Pauli Errors—unwanted applications of $X$, $Y$, or $Z$ gates on a single qubit. In this section, we will build and implement the complete error correction procedure starting from its stabilizer generators:

\begin{align}
S = \langle S_0,\; S_1,\; S_2,\; S_3 \rangle ,\nonumber
\end{align}

with

\begin{align}
S_0 &= X_0 Z_1 Z_2 X_3 I_4,\\[2mm]\nonumber
S_1 &= I_0 X_1 Z_2 Z_3 X_4,\\[2mm]\nonumber
S_2 &= X_0 I_1 X_2 Z_3 Z_4,\\[2mm]\nonumber
S_3 &= Z_0 X_1 I_2 X_3 Z_4.\nonumber
\end{align}
---

## Encoding the logical qubit

First, we need to prepare a data qubit that we want to protect.  
In this tutorial, we will use the qubit  
\begin{align}
|\psi\rangle = \alpha |0\rangle + \beta |1\rangle,\nonumber
\end{align}
as our data qubit.

The next step is to encode this data qubit into a logical qubit. This is done by the **encoding circuit** shown below. Notice that we do not need to know the logical operators to implement the encoding circuit. The circuit is completely determined by the stabilizer generators.  
It is beyond the scope of this tutorial to explain how the circuit is constructed from the stabilizer generators.  
The state after encoding is given by [3]:

\begin{align}
|\tilde{\psi}\rangle = \alpha |\bar{0}\rangle + \beta |\bar{1}\rangle\nonumber
\end{align}
<img src="http://drive.google.com/uc?export=download&id=1cArlK9EzX-wQu8B0yE4iSFWIVNB1lTUL" width="820" height="320"/>

The calculations are a bit cumbersome, but with some patience we can find the common \(+1\)-eigenspace of the stabilizer generators, which are the codewords.

\begin{align}
|\bar{0}\rangle &= \frac{1}{4}\big(
|00000\rangle+ |10010\rangle+ |01001\rangle+ |10100\rangle+ |01010\rangle
- |11011\rangle
- |00110\rangle\nonumber\\
&- |11101\rangle
- |00011\rangle
- |11110\rangle
- |01111\rangle
- |10001\rangle
- |01100\rangle
- |10111\rangle
+ |00101\rangle
\big)\nonumber
\end{align}

\begin{align}
|\bar{1}\rangle = X_1 X_2 X_3 X_4 X_5\, |\bar{0}\rangle.\nonumber
\end{align}

The logical operators bit-flip and phase-flip for this code are  
\begin{align}
\bar{X} = X^{\otimes 5}, \qquad \bar{Z} = Z^{\otimes 5}.\nonumber
\end{align}

Let us implement this encoding circuit in PennyLane.


In [ ]:
def five_qubit_encode(alpha, beta):
    qml.StatePrep([alpha, beta], wires=4)
    qml.Hadamard(wires=0)
    qml.S(wires=0)
    qml.CZ(wires=[0, 1])
    qml.CZ(wires=[0, 3])
    qml.CY(wires=[0, 4])
    qml.Hadamard(wires=1)
    qml.CZ(wires=[1, 2])
    qml.CZ(wires=[1, 3])
    qml.CNOT(wires=[1, 4])
    qml.Hadamard(wires=2)
    qml.CZ(wires=[2, 0])
    qml.CZ(wires=[2, 1])
    qml.CNOT(wires=[2, 4])
    qml.Hadamard(wires=3)
    qml.S(wires=3)
    qml.CZ(wires=[3, 0])
    qml.CZ(wires=[3, 2])
    qml.CY(wires=[3, 4])

## Pauli Errors and Syndrome Measurements

After encoding, the qubit is exposed to noise and decoherence in a real system.  
To simulate this, we introduce an artificial error by randomly acting on one of the physical qubits on wires **0, 1, 2, 3, or 4** with Pauli \(X\), \(Y\), or \(Z\) operations.

Next, we proceed with the **syndrome measurements**, which in this case amounts to applying the **controlled stabilizer generators** on the work wires, as follows:


<img src="http://drive.google.com/uc?export=download&id=12piS0MnKUfSGPZ6Q9L09QxRm4QANoB8F" width="820" height="320"/>

In [ ]:
stabilizers = [
    X(0) @ Z(1) @ Z(2) @ X(3) @ I(4),
    I(0) @ X(1) @ Z(2) @ Z(3) @ X(4),
    X(0) @ I(1) @ X(2) @ Z(3) @ Z(4),
    Z(0) @ X(1) @ I(2) @ X(3) @ Z(4),
]


def five_qubit_error_detection():
    for wire in range(5, 9):
        qml.Hadamard(wires=wire)

    for i in range(len(stabilizers)):
        qml.ctrl(stabilizers[i], control=[i + 5])

    for wire in range(5, 9):
        qml.Hadamard(wires=wire)

We can now combine this with the encoding circuit and the application of the Pauli errors to obtain the circuit that measures the syndrome, as we did in the three-qubit code.

In [ ]:
import pennylane as qml

# Device with shots specified here
dev = qml.device("default.qubit", wires=9, shots=1)

@qml.qnode(dev)
def five_qubit_syndromes(alpha, beta, error_op, error_wire):
    five_qubit_encode(alpha, beta)

    # Apply the error operator
    error_op(wires=error_wire)

    # Measure stabilizers
    five_qubit_error_detection()

    # Return samples from syndrome measurement qubits (5,6,7,8)
    return qml.sample(wires=range(5, 9))

In [ ]:
ops_and_syndromes = []

for wire in (0, 1, 2, 3, 4):
    for error_op in (qml.PauliX, qml.PauliY, qml.PauliZ):
        ops_and_syndromes.append(
            (
                error_op,
                wire,
                five_qubit_syndromes(1 / 2, np.sqrt(3) / 2, error_op, wire),
            )
        )

        print(
            f"{error_op(wire).name[-1]}{wire}",
            five_qubit_syndromes(1 / 2, np.sqrt(3) / 2, error_op, wire),
        )

X0 [[0 0 0 1]]
Y0 [[1 0 1 1]]
Z0 [[1 0 1 0]]
X1 [[1 0 0 0]]
Y1 [[1 1 0 1]]
Z1 [[0 1 0 1]]
X2 [[1 1 0 0]]
Y2 [[1 1 1 0]]
Z2 [[0 0 1 0]]
X3 [[0 1 1 0]]
Y3 [[1 1 1 1]]
Z3 [[1 0 0 1]]
X4 [[0 0 1 1]]
Y4 [[0 1 1 1]]
Z4 [[0 1 0 0]]


The syndrome table is printed, and with it we can apply the necessary operators to fix the corresponding Pauli errors. The script above is straightforward to generalize to any valid set of stabilizers.


## Error correction

The last step is to correct the error by applying the appropriate Pauli operators to the encoded qubits. This time, we have many possible syndrome measurement outcomes. Let us write a helper function to encode the possible syndromes in a way amiable to PennyLane’s mid-circuit measurement capabilities, which only allows for Boolean operators.

Combining all these pieces, we can write the full error correcting code.

In [ ]:
def syndrome_booleans(syndrome, measurements):
    syndrome = qml.math.squeeze(syndrome)
    if syndrome[0] == 0:
        m = ~measurements[0]
    else:
        m = measurements[0]

    for i, elem in enumerate(syndrome[1:]):
        if elem == 0:
            m = m & ~measurements[i + 1]
        else:
            m = m & measurements[i + 1]

    return m

Combining all these pieces, we can write the full error correcting code.

In [ ]:
dev = qml.device("default.qubit", wires=9)


@qml.qnode(dev)
def five_qubit_code(alpha, beta, error_op, error_wire):
    five_qubit_encode(alpha, beta)

    error_op(wires=error_wire)

    five_qubit_error_detection()

    m5 = qml.measure(5)
    m6 = qml.measure(6)
    m7 = qml.measure(7)
    m8 = qml.measure(8)

    measurements = [m5, m6, m7, m8]

    for op, wire, synd in ops_and_syndromes:
        qml.cond(syndrome_booleans(synd, measurements), op)(wires=wire)

    return qml.density_matrix(wires=[0, 1, 2, 3, 4])

Let us check that the fidelity between the output state and the initial encoded state is equal to 1 for arbitrary Pauli errors on one qubit. Indeed:

In [ ]:
@qml.qnode(qml.device("default.qubit", wires=5))
def five_qubit_encoded_state(alpha, beta):
    five_qubit_encode(alpha, beta)
    return qml.state()


encoded_state = qml.math.dm_from_state_vector(five_qubit_encoded_state(alpha, beta))
for wire in range(5):
    for error_op in (qml.PauliX, qml.PauliY, qml.PauliZ):
        print(
            f"Fidelity when error {error_op(wire).name[-1]}{wire}:",
            qml.math.fidelity(
                encoded_state, five_qubit_code(alpha, beta, error_op, wire)
            ).round(2),
        )

Fidelity when error X0: 1.0
Fidelity when error Y0: 1.0
Fidelity when error Z0: 1.0
Fidelity when error X1: 1.0
Fidelity when error Y1: 1.0
Fidelity when error Z1: 1.0
Fidelity when error X2: 1.0
Fidelity when error Y2: 1.0
Fidelity when error Z2: 1.0
Fidelity when error X3: 1.0
Fidelity when error Y3: 1.0
Fidelity when error Z3: 1.0
Fidelity when error X4: 1.0
Fidelity when error Y4: 1.0
Fidelity when error Z4: 1.0


The fidelity is 1.0 after error correction, which means the output state is the same!

Note that to build the encoding, syndrome measurement, and error correction circuits, we did only use the stabilizer generators. This is a powerful feature of the stabilizer formalism. It allows us to construct the code from its stabilizer generators and then use the code to correct errors. However, we can also find the codewords and logical operators directly from the stabilizer generators by finding the common +1-eigenspace of the stabilizer generators.